In [1]:
import pandas as pd

/home/am/.local/lib/python3.6/site-packages/pandas/compat/_optional.py:106: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.2.0' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
sth = [0,2]
archer = lambda x: [x[c] for c in sth]
archer(['a', 'b', 'c'] )

['a', 'c']

In [3]:
from shapely.geometry import Polygon
def polygonize(p):
    polygons = []
    for row in range(0,p.shape[0]):
        poly = list(map(float, p.iloc[row]))
        xy = ()
        for i in range(len(poly)):
            if(i%2 == 0):
                xy += (((poly[i+1]), poly[i]),)
        polygons.append(str(Polygon(xy)))
    return pd.DataFrame(polygons)

In [4]:
geometryCols = ['Center Latitude dec', 'Center Longitude dec', 'NW  Corner Lat dec',
       'NW Corner Long dec', 'NE Corner Lat dec', 'NE Corner Long dec',
       'SE Corner Lat dec', 'SE Corner Long dec', 'SW Corner Lat dec',
       'SW Corner Long dec']

sentMetaData = pd.read_csv('SENTINEL_2A_382286.csv',
                  usecols=['Acquisition End Date', 'Tile Number', 'Cloud Cover', 'Center Latitude dec', 'Center Longitude dec', 'NW  Corner Lat dec',
       'NW Corner Long dec', 'NE Corner Lat dec', 'NE Corner Long dec',
       'SE Corner Lat dec', 'SE Corner Long dec', 'SW Corner Lat dec',
       'SW Corner Long dec'], 
                  parse_dates=['Acquisition End Date'], 
                  date_parser = lambda x: pd.to_datetime(x).date
                  ).assign(Geometry = lambda x: polygonize(pd.DataFrame(x[geometryCols])) ).drop(columns = geometryCols)
sentMetaData = sentMetaData.rename(columns={'Acquisition End Date':'Date'})
print(sentMetaData.columns)
sentMetaData.head()

Index(['Date', 'Tile Number', 'Cloud Cover', 'Geometry'], dtype='object')


,Date,Tile Number,Cloud Cover,Geometry
0,2019-01-01,T37NGE,2.0074,"POLYGON ((41.2953727 4.024012399999999, 40.802..."
1,2019-01-01,T37NFH,0.0781,"POLYGON ((40.4015308 6.7391924, 39.9058064 7.2..."
2,2019-01-01,T38NKP,0.0000,"POLYGON ((42.7781127 7.639713, 42.2775766 8.13..."
3,2019-01-01,T38PLS,2.3715,"POLYGON ((43.67468289999999 10.3562239, 43.170..."
4,2019-01-01,T37NFE,5.4055,"POLYGON ((40.3953304 4.0260565, 39.9014269 4.5..."


In [5]:
sentMetaData['Geometry'][0]

'POLYGON ((41.2953727 4.024012399999999, 40.8022674 4.5216809, 41.7912577 4.518543599999999, 41.7878806 3.5263828, 40.8000849 3.5288293, 41.2953727 4.024012399999999))'

### Set Multi-indices 'Date' and 'Tiles'
###### abandoned due to inaccessibility

In [6]:
# print(sentMetaData['Date'].year)
sentMetaDataMonthly = sentMetaData.assign(Month= sentMetaData['Date'].dt.strftime('%m-%Y') ).drop(columns = 'Date')

In [7]:
print(sentMetaDataMonthly.columns)
# sentMetaData.loc[sentMetaData['Month']==12]

sentMetaDataMonthly

Index(['Tile Number', 'Cloud Cover', 'Geometry', 'Month'], dtype='object')


,Tile Number,Cloud Cover,Geometry,Month
0,T37NGE,2.0074,"POLYGON ((41.2953727 4.024012399999999, 40.802...",01-2019
1,T37NFH,0.0781,"POLYGON ((40.4015308 6.7391924, 39.9058064 7.2...",01-2019
2,T38NKP,0.0000,"POLYGON ((42.7781127 7.639713, 42.2775766 8.13...",01-2019
3,T38PLS,2.3715,"POLYGON ((43.67468289999999 10.3562239, 43.170...",01-2019
4,T37NFE,5.4055,"POLYGON ((40.3953304 4.0260565, 39.9014269 4.5...",01-2019
...,...,...,...,...
18141,T37NGG,0.0000,"POLYGON ((41.3015875 5.831693700000001, 40.807...",01-2018
18142,T38NKK,0.0000,"POLYGON ((42.7923049 4.0242553, 42.29644279999...",01-2018
18143,T37PHK,0.0000,"POLYGON ((42.2230716 8.536749499999999, 41.728...",01-2018
18144,T37PHL,0.3930,"POLYGON ((42.2310749 9.4399961, 41.7356727 9.9...",01-2018


In [8]:
monthlySMD = sentMetaDataMonthly.groupby('Tile Number')
monthlySMD.head()

,Tile Number,Cloud Cover,Geometry,Month
0,T37NGE,2.0074,"POLYGON ((41.2953727 4.024012399999999, 40.802...",01-2019
1,T37NFH,0.0781,"POLYGON ((40.4015308 6.7391924, 39.9058064 7.2...",01-2019
2,T38NKP,0.0000,"POLYGON ((42.7781127 7.639713, 42.2775766 8.13...",01-2019
3,T38PLS,2.3715,"POLYGON ((43.67468289999999 10.3562239, 43.170...",01-2019
4,T37NFE,5.4055,"POLYGON ((40.3953304 4.0260565, 39.9014269 4.5...",01-2019
...,...,...,...,...
1405,T37PCN,11.1592,"POLYGON ((37.6707083 11.2605722, 37.164699 11....",12-2018
1409,T37PCP,3.9885,"POLYGON ((37.6663787 12.1648642, 37.158462 12....",12-2018
1412,T36NZK,9.8253,"POLYGON ((36.19538170000001 4.020972, 35.70319...",12-2018
1425,T36NZM,26.0208,"POLYGON ((36.2040245 5.827279799999999, 35.711...",12-2018


In [9]:
# print(sentMetaDataMonthly['Month'].unique)
monthlySMD = sentMetaDataMonthly.groupby(['Month','Tile Number'])['Cloud Cover'].mean()

In [ ]:
# print(sentMetaDataMonthly.columns)
# print(sentMetaDataMonthly.index.names[0])
# monthlySMD = lambda df: df.xs(month for month in df['Month'].unique()  ).groupby('Tile Number')
# monthlySMD(sentMetaDataMonthly)

In [11]:
import geopandas as gpd

smdGPD = gpd.GeoDataFrame(sentMetaDataMonthly.groupby(['Month','Tile Number','Geometry'])['Cloud Cover'].mean()) 
print(smdGPD.index.names)
smdGPD.head()
smdGPD.columns
smdGPD.head()
# monthlySMD = lambda df: df.xs(month for month in df['Month'].unique()  ).groupby('Tile Number').mean() 
# monthlySMD(smdGPD)

['Month', 'Tile Number', 'Geometry']


Cloud Cover
Month   Tile Number Geometry                                                       
01-2018 T36NVP      POLYGON ((32.5907275 7.6452367, 32.0919113 8.14...    11.176215
        T36NWN      POLYGON ((33.4965969 6.740951299999999, 32.9998...     7.033814
        T36NWP      POLYGON ((33.4975804 7.6451439, 32.9998184 8.14...    11.231400
        T36NXM      POLYGON ((34.3991109 5.8346612, 33.9041175 6.33...     0.537750
        T36NXN      POLYGON ((34.4015308 6.7391924, 33.9058064 7.23...    10.827314

In [ ]:
import matplotlib.pyplot as plt
plt.plot(smdGPD['Geometry'])

In [ ]:
# monthlySMDGPD = 
# smdGPD.groupby('Month')['Cloud Cover'].mean().head()
print(smdGPD['Tile Number'].unique())
pd.MultiIndex.from_frame(smdGPD, names=['Month'])